### <center> NLP - TEXT MININING PROJECT | Fernando Llopis Bernal </center>

Librerias que voy a utilizar

In [1]:
import numpy as np
import pandas as pd
import re

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim import models
import itertools

from nltk import pos_tag
import nltk

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

<html lang="es"><head><title>Paro en España: La crisis del coronavirus provoca la pérdida de 900.000 empleos desde el inicio del estado de alarma | Economía | EL PAÍS</title><meta name="lang" content="es"><meta name="author" content="Manuel V. Gómez"><meta name="robots" content="index,follow"><meta name="description" content="Los efectos de la Covid-19 dan un golpe sin precedentes al mercado laboral y elevan la cifra de parados a 3,5 millones"><meta name="news_keywords" content="Economía, Crisis económica, Crisis económica coronavirus covid-19, Paro registrado, Afiliación Seguridad Social, Mercado laboral, Sociedad, Coronavirus Covid-19, Coronavirus, Desempleo, Parados, Empleo"><meta name="date" scheme="W3CDTF" content="2020-04-02T06:59:36.775Z"><meta name="organization" content="Ediciones EL PAÍS S.L."><meta property="article:modified_time" content="2020-04-02T11:36:15.907Z"><meta property="article:published_time" content="2020-04-02T06:59:36.775Z"><meta property="article:publisher" c

unable to import 'smart_open.gcs', disabling that module


A continuación, configuro los parámetros para la visualización de los varios dataframes en mi PyCharm, que es donde lo he desarrollado:

In [2]:
# Dataframe visualization parameters
desired_width = 320
pd.set_option('display.width', desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 30)
pd.options.display.float_format = '{:.3f}'.format

In [3]:
train_df = pd.read_csv('/Users/fer/Downloads/train.csv')
test_df = pd.read_csv('/Users/fer/Downloads/test.csv')
sample_df = pd.read_csv('/Users/fer/Downloads/sample_submission.csv')

In [4]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [6]:
train_df.describe()

,id,target
count,7613.000,7613.000
mean,5441.935,0.430
std,3137.116,0.495
min,1.000,0.000
25%,2734.000,0.000
50%,5408.000,0.000
75%,8146.000,1.000
max,10873.000,1.000


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
id          3263 non-null int64
keyword     3237 non-null object
location    2158 non-null object
text        3263 non-null object
dtypes: int64(1), object(3)
memory usage: 102.0+ KB


In [9]:
test_df.describe()

,id
count,3263.000
mean,5427.153
std,3146.427
min,0.000
25%,2683.000
50%,5500.000
75%,8176.000
max,10875.000


Lo primero que he pretendido a continuación, creando dos dataframes para realizar un análisis exploratorio, es ver si podrían ser representativos sólo aquellas observaciones que contengan en última instancia, ambas observaciones (no NaN, NA, ...) y posteriormente ver si uso esos subsets o ambas poblaciones enteras:

In [10]:
nas_keyword = sum(pd.isnull(train_df['keyword']))
print('nas keyword: ', nas_keyword)
nas_location = sum((pd.isnull(train_df['location'])))
print('nas location: ', nas_location)

nas keyword:  61
nas location:  2533


In [11]:
nas_keyword = sum(pd.isnull(test_df['keyword']))
print('nas keyword: ',nas_keyword)
nas_location = sum((pd.isnull(test_df['location'])))
print('nas location: ', nas_location)

nas keyword:  26
nas location:  1105


In [12]:
''' Subset keyword and location columns :: for != to NaN'''
train_k = train_df[train_df.keyword.notnull()].reset_index()
print('Train set with all observations containing any valid keyword\n', train_k.head())
train_l = train_df[train_df.location.notnull()].reset_index()
print('Train set with all observations containing any valid location\n', train_l.head())
train_kl = train_df[(train_df.keyword.notnull()) & (train_df.location.notnull())].reset_index()
test_dfx = test_df[(test_df.keyword.notnull()) & (test_df.location.notnull())].reset_index()

Train set with all observations containing any valid keyword
    index  id keyword                       location                                                                                text  target
0     31  48  ablaze                     Birmingham                             @bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C       1
1     32  49  ablaze  Est. September 2012 - Bristol                 We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw       0
2     33  50  ablaze                         AFRICA  #AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi       1
3     34  52  ablaze               Philadelphia, PA                                                  Crying out for more! Set me ablaze       0
4     35  53  ablaze                     London, UK        On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE http://t.co/qqsmshaJ3N       0
Train set with all observations containing any valid location
    index  i

### Extracción de conocimiento mediante expresiones regulares (RegEx)

A continuación, realizo en primera instancia una extracción de los conceptos que me parecen o me pueden parecer interesantes desde un punto de vista 'humano', como los hashtags -que visualmente me han parecido señales para predecir si es un desastre real o no- y los usuarios -que a priori, no me han parecido relevantes para posteriormente entrenar los modelos de Machine Learning que he seleccionado, pero que tampoco quiero descartar, prefiriendo que esta decisión la tomen los algoritmos-.

Realizo los procesos para ambos sets de datos, el de entrenamiento y el de test:

In [13]:
''' RegEx '''
print(train_kl.text)
hashtags = re.findall(r'#.+?\s', string=str(train_kl.text))
hashtags_t = re.findall(r'#.+?\s', string=str(test_df.text))
hashtags = [re.findall(r'#.+?\s', item) for item in train_df.text]
hashtags_t = [re.findall(r'#.+?\s', item) for item in test_df.text]
print('length of hashtags: ', len(hashtags))

hashtags = []
for item in train_df.text:
    loquesea = re.findall(r'#.+?\s', item)
    hashtags.append(loquesea)

users = [re.findall(r'@.+?\s', item) for item in train_df.text]
users_t = [re.findall(r'@.+?\s', item) for item in test_df.text]
print('lenght of users: ', len(users))

# add hashtags and users to the main dataframe:
train_df['hashtags'] = [item[0] if type(item) == 'list' else item for item in hashtags]
train_df['users'] = [item[0] if type(item) == 'list' else item for item in users]
test_df['hashtags'] = [item[0] if type(item) == 'list' else item for item in hashtags_t]
test_df['users'] = [item[0] if type(item) == 'list' else item for item in users_t]


0                                                   @bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
1                                       We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw
2                        #AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi
3                                                                        Crying out for more! Set me ablaze
4                              On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE http://t.co/qqsmshaJ3N
5       @PhDSquares #mufc they've built so much hype around new acquisitions but I doubt they will set t...
6                                                   INEC Office in Abia Set Ablaze - http://t.co/3ImaomknnA
7                                                                                    Ablaze for you Lord :D
8       Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.c...
9       Had an awesome time 

A continuación, convierto cada tweet en frases con la librería de NLTK, con sent_tokenize. Posteriormente, sobre las mismas frases y para cada observación, transformo cada frase compuesta, en tokens y finalmente en tokens alfanuméricos con el método .isalpha(), ya que sólo me interesan ese tipo de palabras (con o sin números) y no los símbolos (la semántica pura, de cara a extraer el conocimiento necesario para los modelos predictivos):

In [14]:
train_tokenization = 1
if train_tokenization == 1:

    sentences = [sent_tokenize(sentence) for sentence in train_df.text]
    print('train text sencences: ', len(sentences))

    tokens_per_sent = [word_tokenize(str(t).lower()) for t in sentences]
    print('train tokens_per_sent: ', len(tokens_per_sent))

    tokens = []
    for sentence in tokens_per_sent:
        logic = []
        tokens.append(logic)
        for word in sentence:
            if word.isalpha():
                logic.append(word)

    train_df['tokens'] = tokens

test_tokenization = 1
if test_tokenization == 1:

    sentences = [sent_tokenize(sentence) for sentence in test_df.text]
    print('test text sentences: ', len(sentences))

    tokens_per_sent = [word_tokenize(str(t).lower()) for t in sentences]
    print('test tokens_per_sent: ', len(tokens_per_sent))

    tokens = []
    for sentence in tokens_per_sent:
        logic = []
        tokens.append(logic)
        for word in sentence:
            if word.isalpha():
                logic.append(word)

    test_df['tokens'] = tokens

train text sencences:  7613
train tokens_per_sent:  7613
test text sentences:  3263
test tokens_per_sent:  3263


In [15]:
train_df.head()

,id,keyword,location,text,target,hashtags,users,tokens
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,[#earthquake ],[],"[deeds, are, the, reason, of, this, earthquake, may, allah, forgive, us, all]"
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,[],[],"[fire, near, la, ronge, sask]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or...,1,[],[],"[all, residents, asked, to, in, place, are, being, notified, by, officers, other, evacuation, or..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,[#wildfires ],[],"[people, receive, wildfires, evacuation, orders, in, california]"
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,"[#Alaska , #wildfires ]",[],"[got, sent, this, photo, from, ruby, alaska, as, smoke, from, wildfires, pours, into, a, school]"


In [16]:
test_df.head()

,id,keyword,location,text,hashtags,users,tokens
0,0,NaN,NaN,Just happened a terrible car crash,[],[],"[happened, a, terrible, car, crash]"
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone.",[#earthquake ],[],"[about, earthquake, is, different, cities, stay, safe, everyone]"
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",[],[],"[is, a, forest, fire, at, spot, pond, geese, are, fleeing, across, the, street, i, can, not, sav..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,[#Spokane ],[],"[lighting, spokane, wildfires]"
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,[],[],"[soudelor, kills, in, china, and, taiwan]"


A continuación, creo un corpus con todos los tokens que me interesan transformados a números (id) con su frecuencia (en tuples en el formato: (id, frecuencia)). 

Una vez realizada esa tarea, paso para cada una de las observaciones -para cada lista de tokens de cada observacion, y que he añadido previamente como nueva variable a cada dataframe (train y test)-, paso el peso relativo a cada token en cada lista y así ponderar cada valor en cada tweet.

Por último, mediante el método pos_tag() de la libreria NLTK, extraigo en listas de tuples, el tipo de token (si es verbo, nombre propio, etc.). Mi objetivo aquí es, mediante el uso de los algoritmos de Machine Learning propuestos más adelante, ver si ese conocimiento, que *aún no he puesto en variables a los train y test sets porque no sé bien cómo (desde el punto de vista lingüistico, no técnico)*. Me está costando mucho esa abstracción.

In [17]:
dictionary = Dictionary(train_df.tokens)
corpus = dictionary.doc2bow(list(itertools.chain(*train_df.tokens))) # bow

tfidf = TfidfModel([corpus])
tfidf_weights = tfidf[corpus]

doc_weight = []
for tweet in train_df.tokens:
    for word in tweet:
        dictionary.token2id.get(word)

nltk.download('averaged_perceptron_tagger')

tagged_tokens = [pos_tag(tag) for tag in train_df.tokens]
print('Length of tagged tokens: ', len(tagged_tokens))

train_df['tagged'] = tagged_tokens

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/fer/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Length of tagged tokens:  7613


# PROBLEMA

**Aquí es donde sale mi verdadero problema:** sé como transformar, mediante preprocesado de variables categoricas, transponiendo cada valor único de cada observacion de una variable específica en nombres de variables con valor igual a 1 para aquellas concidencias de par fila-columna y 0 para las restantes (usando el método de One-Hot Enconding => ohe). Lo que no sé, y me ha llevado muchas horas sin avanzar nada, -y me gustaría pedirte ayuda o consejo para cómo actuar en estas situaciones-, es si crear una valor de ohe para cada frase, parabra, sigificado, grupos de n-grams ... no sé que tiene o tendría sentido hacer ... De verdad que me desespera porque me cuesta entenderlo.

Posteriormente, pretendía y no me ha dado tiempo y te pido disculpas por ello, transformar el resto de valores que no sean categoricos, y dependiendo del contexto eliminarlos o no ... mediante normalización -valor entre (0, 1)- y estandarización para poder comparar valores en mismas escalas. Para ello pretendía seleccionar los algoritmos de preprocesado: MinMaxScaler() y StandardScaler().

In [18]:
categorical_encoding = 0
if categorical_encoding == 1:

    ohe = OneHotEncoder()
    X_train_df = ohe.fit(train_df.dropna())
    y_train = train_df.target

    X_test_df = ohe.fit(test_df.dropna())
    y_test = test_df.target


En este último paso, una vez tenga los sets de datos de entrenamiento y test (este último como hold-out set al usar un Cross-Validation previo al training set), evaluar con unos buenos parámetros para cada modelo -**he seleccionado como propones, un algoritmo de Support Vector Machine y también como clásico método, una Regresión Lógistica**- cuál es el mejor algoritmo para este caso de uso.

En realidad me hubiera gustado mucho meter un último modelo, y realizar un Ensemble con un Voting Classifier y ver por este mismo método democrático, si quedarnos con 1 <'real disaster'> ó 0 <'not real'> en función de esa votación (pero claro, no he pasado del encoder ...):

In [19]:
grid_method = 0
if grid_method == 1:

    classification_hold_out = 0
    if classification_hold_out == 1:

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        logreg_activate = 0
        if logreg_activate == 1:

            c_space = np.logspace(-5, 8, 15)
            param_grid = {'C': c_space}

            logreg = LogisticRegression()
            logreg_cv = GridSearchCV(estimator=logreg, param_grid=param_grid, scoring='accuracy', cv=5)
            logreg_cv.fit(X_train, y_train)

            best_params = logreg_cv.best_params_
            best_score = logreg_cv.best_score_
            print('\nOptimal tuned params (\'C\' in this case): ', best_params)
            print('Optimal tuned score (accuracy): ', best_score)

        svm_activate = 0
        if svm_activate == 1:

            svm_model = svm.SVC(kernel='linear')  # instantiate
            svm_model.fit(X_train, y_train)
            y_pred_svm_model = svm_model.predict(X_test)
            print('This is my linear model:\n', y_pred_svm_model)

            print('Accuracy: {:.2f}'.format(metrics.accuracy_score(y_test, y_pred_svm_model)))
            print('Precision: {:.2f}'.format(metrics.precision_score(y_test, y_pred_svm_model)))
            print('Recall: {:.2f}'.format(metrics.recall_score(y_test, y_pred_svm_model)))